In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [2]:
# from langchain_text_splitters import NLTKTextSplitter, SpacyTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (BSHTMLLoader, 
                                                  DirectoryLoader, 
                                                  Docx2txtLoader, 
                                                  NewsURLLoader, 
                                                  PyPDFLoader, 
                                                  PyMuPDFLoader,
                                                  MathpixPDFLoader,
                                                  RecursiveUrlLoader, 
                                                  SeleniumURLLoader, 
                                                  TextLoader, 
                                                  UnstructuredHTMLLoader,
                                                UnstructuredImageLoader,
                                                UnstructuredPowerPointLoader, 
                                                UnstructuredURLLoader, 
                                                UnstructuredWordDocumentLoader, 
                                                YoutubeLoader)
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter

from bs4 import BeautifulSoup as Soup
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from unstructured.cleaners.core import clean_extra_whitespace
import os
import torch
import re
from PIL import Image

from transformers import NougatProcessor, VisionEncoderDecoderModel, pipeline
from huggingface_hub import hf_hub_download

from datasets import load_dataset

from src.utils import *

In [3]:
data = config_parser(data_config_path = 'config/data_config.yaml')

In [4]:
single_url_list = data.get('single_url_list')
loader = NewsURLLoader(urls=single_url_list, 
                    post_processors=[clean_extra_whitespace],)
ori_text = loader.load()


In [17]:
pdf_path = "data/web_data/Chapter 0-Course introduction.pdf"
loader = PyMuPDFLoader(pdf_path)
ori_text = loader.load()
ori_text = combine_short_doc(ori_text, 100)


In [6]:
ori_text[10]

Document(page_content='1987: THE FAST MULTIPOLE METHOD\nIn 4 billion years time…\n HOW DOES IT WORK?\nGreengard\nRokhlin\n HOW DO I USE IT?\nA\n=\n+\n+\n+ · · ·\nFull rank form\nLow rank form\nOriginal\nrank 1\nrank 3\nrank 10\nrank 50\nThe low rank format saves computational time and storage costs\nThe SVD gives the best low rank approximations:\n', metadata={'source': 'data/web_data/TopTenAlgorithms.pdf', 'file_path': 'data/web_data/TopTenAlgorithms.pdf', 'page': 16, 'total_pages': 20, 'format': 'PDF 1.4', 'title': 'TopTenAlgorithms.key', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Keynote', 'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creationDate': "D:20170325100036Z00'00'", 'modDate': "D:20170325100036Z00'00'", 'trapped': ''})

In [23]:
processor = NougatProcessor.from_pretrained("facebook/nougat-base")
model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-base")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# prepare PDF image for the model
filepath = "data/web_data/image/slide1.jpg"
image = Image.open(filepath).convert("RGB")

In [43]:
pixel_values = processor(image, return_tensors="pt").pixel_values

# generate transcription (here we only generate 30 tokens)
outputs = model.generate(
    pixel_values.to(device),
    min_length=1,
    max_new_tokens=1024,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
)

sequence = processor.batch_decode(outputs, skip_special_tokens=True)
print(repr(sequence))


['**Quicksort\'s Average Running Time Analysis**\n\n* In order to have a good "average performance," one can _randomize_ this algorithm by assuming that each pivot is chosen at random.\n* Let us compute the expectation of the number \\(X\\) of comparisons needed when running the randomized version of quicksort.\n* Recall that the input is a sequence \\(S\\) = \\((x_{1},...,x_{n})\\) of distinct elements, and that \\((y_{1},...,y_{n})\\) has the same elements sorted in increasing order.\n* In order to compute \\(E(X)\\), we decompose \\(X\\) as a sum of indicator variables \\(X_{i,j}\\), with \\(X_{i,j}\\) = 1 iff \\(y_{i}\\) and \\(y_{j}\\) are ever compared, and \\(X_{i,j}\\) = 0 otherwise.\n* Then, it is clear that \\(X\\) = \\(\\sum_{j=2}^{n}\\sum_{i=1}^{j-1}X_{i,j}\\) and \\(E(X)\\) = \\(\\sum_{j=2}^{n}\\sum_{i=1}^{j-1}E(X_{i,j})\\).\n* Furthermore, since \\(X_{i,j}\\) is an indicator variable, we have \\(E(X_{i,j})\\) = \\(P(y_{i}\\) and \\(y_{j}\\) are ever compared).']


In [37]:
text = (repr(sequence)[1:-1])

In [57]:

# recursive_url_list = data.get('recursive_url_list')

# loader = RecursiveUrlLoader(
#     url=recursive_url_list[0], max_depth=2,
#       extractor=lambda x: Soup(x, "html.parser").text,
# )
# docs = loader.load()

In [58]:
# loader = Docx2txtLoader("web_data/sample4.docx")
# data = loader.load()
# print(data)

In [59]:
# urls = [
# "https://e.vnexpress.net/news/travel-guide/binh-phuoc-boasts-jungle-treks-charming-feasts-4673392.html",
# 'https://e.vnexpress.net/news/culture/vietnamese-comic-artist-wins-silver-at-int-l-manga-awards-4695364.html',
# ]
# loader = UnstructuredURLLoader(urls=urls, 
#                     post_processors=[clean_extra_whitespace],)
# data = loader.load()
# print(data)
#

In [60]:
# youtube_url_list = data.get('youtube_url_list')

# loader = YoutubeLoader.from_youtube_url(
#     youtube_url_list[0], add_video_info=False
# )
# data = loader.load()
# print(data)

In [6]:
   
class NewsSummarizer:
    def __init__(self, summarizer = pipeline("summarization", 
                                             model="Falconsai/text_summarization", 
                                             torch_dtype=torch.float16,
                                             device = take_device()),
                #  translator = GoogleTranslator(),
                 chunk_overlap:str = 10,
                 max_length:int=200, 
                 min_length:int=30,
                 ):
        self.summarizer = summarizer
        # self.translator = translator
        self.max_length = max_length
        self.min_length = min_length
        self.chunk_overlap = chunk_overlap
        self.text_splitter = self.load_text_splitter()
    
    def load_text_splitter(self):
        text_splitter =  NLTKTextSplitter(chunk_size=1000)
        return text_splitter

    def summary_text(self,text_chunks:list):
        '''Summary short text'''
        sum_text= f''
        for model_output in self.summarizer(text_chunks, batch_size=8, 
                                            truncation="only_first",):
            text = model_output['summary_text']
            sum_text += f'\n{text}'
        return sum_text
    
    def summary_news(self, news:str)->str:
        # trans_news = self.translator.translate(text=news, to_lang='en')
        text_chunks = self.text_splitter.split_text(news)
        summary_text = self.summary_text(text_chunks)
        # summary_text = self.translator.translate(text=summary_text, to_lang='vi')

        return summary_text


Selected device: cuda


In [7]:
text = ori_text[0].page_content

In [8]:
text

'THE TOP10 ALGORITHMS FROM \nTHE 20TH CENTURY\nAlex Townsend\nCornell University\n THE TOP10 LIST\n1946: The Metropolis Algorithm \n1947: Simplex Method \n1950: Krylov Subspace Method\n1951: The Decompositional Approach to Matrix Computations \n1957: The Fortran Optimizing Compiler\n1959: QR Algorithm \n1962: Quicksort \n1965: Fast Fourier Transform\n1977: Integer Relation Detection  \n1987: Fast Multipole Method\nDantzig von Neumann Hestenes Householder\nBackus\nHoare\nGreengard\n'

In [9]:
len(ori_text)

18

In [18]:
for i, txt in enumerate(ori_text[:5]):
    print(f"{txt.page_content}\n-------")

1
DATA ENGINEERING
COURSE OVERVIEW
Trong Nhan Phan, PhD
 CONTACT
❑Lecturer: Phan Trọng Nhân, PhD
❑Faculty: Computer Science and Engineering
❑Department: Information Systems
❑Email: nhanpt@hcmut.edu.vn
❑Course site: LMS
2

-------
COURSE INTRODUCTION
❑Subject: Data Engineering
❑Number: 055240
❑Credit: 3
❑Period: 70.5
❑Prerequisite: None
❑Class Periods: 13 weeks
❑Course Syllabus
3

-------
COURSE AIMS
◼Database analysis and design methodologies
including
relational
data
model
and
NoSQL
models.
◼Advances in data storage and retrieval methods,
flexible query answering, query processing and
optimization, transaction processing.
◼Object-oriented and time series databases, as
well as big data and applications.
◼Research topics relevant to modern databases
and applications will also be introduced and
discussed.
4

-------
REFERENCES
[1]
R.
Elmasri,
S.B.
Navathe:
“Fundamentals
of
Database
Systems”, 7th Edition, Pearson Addison-Wesley, 2016.
[2]
P.A.
Bernstein,
E.
Newcomer:
"Principles
of
Transa

In [11]:
def check(texts):
    for i, t in enumerate(texts):
        print(f"{texts[i]}\n-----------")

In [12]:
ori_text[12]

Document(page_content='HOW DO I USE IT?\nAn automatic way to tell us how “complicated” a function is.\nFFT\n', metadata={'source': 'data/web_data/TopTenAlgorithms.pdf', 'file_path': 'data/web_data/TopTenAlgorithms.pdf', 'page': 12, 'total_pages': 20, 'format': 'PDF 1.4', 'title': 'TopTenAlgorithms.key', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Keynote', 'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creationDate': "D:20170325100036Z00'00'", 'modDate': "D:20170325100036Z00'00'", 'trapped': ''})

In [13]:
text = ori_text[12].page_content
print(text)

HOW DO I USE IT?
An automatic way to tell us how “complicated” a function is.
FFT



In [14]:
token_text_splitter = TokenTextSplitter(chunk_size=150, chunk_overlap=10)
token_texts = token_text_splitter.split_text(text)
check(token_texts)

HOW DO I USE IT?
An automatic way to tell us how “complicated” a function is.
FFT

-----------


In [15]:
nltk_text_splitter = NLTKTextSplitter(chunk_size=1000, chunk_overlap=100)
nltk_texts = nltk_text_splitter.split_text(text)
check(nltk_texts)

HOW DO I USE IT?

An automatic way to tell us how “complicated” a function is.

FFT
-----------


In [16]:
def sum_check(text_chunks):
    sum_text = NewsSummarizer().summary_text(text_chunks)
    print(sum_text)


In [17]:
sum_check(token_texts)

Your max_length is set to 200, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)



FFT is an automatic way to tell us how “complicated” a function is . FFT has a simple function that can be accessed by a user .


In [18]:
sum_check(nltk_texts)

Your max_length is set to 200, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)



FFT is an automatic way to tell us how “complicated” a function is . FFT has a simple function that can be accessed by a user .
